# Particle Physics Coursework 2

# Introduction

### The purpose of this exercise is to gain a deeper understanding of the electroweak interaction. We will use real data from the CMS experiment at the LHC to extract the weak mixing angle $\sin^2\theta_W$, one of the fundamental parameters of the Standard Model. This will be achieved by fitting a theoretical expression for the forward-backward asymmetry $A_{FB}$ of di-muon pairs to a measured $A_{FB}$ distribution obtained from data.

This coursework will require you to download all files contained in the following link **[[Input files]](https://uob-my.sharepoint.com/:f:/g/personal/kp14102_bristol_ac_uk/EogitLtkOqZBleDd2RR8fF4B6KfO_X4jzsSkp2ekYHHATw?e=9mNFso)** and place **ALL** these files in the same directory as this notebook.

Just as in the previous exercise, we will analyse events in which a Z boson was produced that subsequently decays to a pair of muons. The data [1,2] has been preselected for you and can be found in the file `cms_muons.root`.

The measurement we are about to carry out follows (*with some simplifications!*) what is described **in this [CMS paper](https://link.springer.com/article/10.1140/epjc/s10052-018-6148-7)** [3]. In particular the introduction of this paper describes how the forward-backward asymmetry $A_{FB}$ of the di-muon pairs is related to the vector- and axial-vector couplings, and thus to $\sin^2\theta_W$. **As a minimum, you will need to read the introduction of this paper but other sections will be helpful as well.** 

For some theoretical background on the electroweak mixing angle, take a look at Chapters 15.3 through to 16.2 in the textbook *Modern Particle Physics* by Thomson (Cambridge University Press). **In particular Chapter 16.2.2 is relevant for this exercise** but a key difference is that it discusses this measurement at an electron-positron collider, and not at a hadron collider such as the LHC.  

Chapter 9.7 (in particular 9.7.3) in the textbook by Griffiths has a very brief introduction to this topic.

References:

[1] Wunsch, Stefan; (2021). Run2012B_DoubleMuParked dataset in reduced NanoAOD format for education and outreach. CERN Open Data Portal. 
DOI:10.7483/OPENDATA.CMS.04XV.ESBR

[2] Wunsch, Stefan; (2021). Run2012C_DoubleMuParked dataset in reduced NanoAOD format for education and outreach. CERN Open Data Portal. DOI:10.7483/OPENDATA.CMS.86ZS.7G78

[3] The CMS Collaboration, Measurement of the weak mixing angle using the forward-backward asymmetry of Drell-Yan events in pp collisions at 8 TeV, Eur. Phys. J. C 78 (2018) 701, DOI: https://doi.org/10.1140/epjc/s10052-018-6148-7


# Question 1 (5 marks)
Unlike at LEP, where the polar angle of the scattered muons is measured with respect to the incoming electron-positron pair, we need to consider the directions and momenta of the quark-antiquark pair at a hadron collider such as the LHC. In general, as a result of different incoming quark and anti-quark momenta, the rest frame of the produced Z boson is not the same as the laboratory frame.

We can define the scattering angle $\cos\theta^*$ as the cosine of the polar angle of the negatively charged muon ($\mu^-$) in the vector boson’s rest frame (the Collins–Soper frame). This measures how “forward” ($\cos\theta^* > 0$) or “backward” ($\cos\theta^* < 0$) the lepton is emitted relative to the incoming quark direction.

- Give the expression for the angle $\cos\theta^*$ in the Collins-Soper frame and define all the quantities appearing in the formula.

# Question 2 (5 marks)

Experimentally, the forward-backward asymmetry $A_{FB}$ is given by the difference in the forward and backward cross-sections, divided by their sum: $A_{FB} = \frac{\sigma_{F} - \sigma_{B}}{\sigma_{F} + \sigma_{B}}$. This quantity is sensitive to the vector and axial-vector couplings of electroweak bosons to fermions. The forward and backward cross-sections are proportional to the number of events with leptons in the forward and backward direction (in the Collins-Soper frame).

- State here the equation that connects the forward-backward asymmetry as a function of the di-muon invariant mass, $A_{FB} (M)$, to the vector- and axial-vector couplings ($v_f$ and $a_f$) and write down their definitions. Also define all quantities that appear in this equation.

## Getting started with the CMS data

To begin with we need to import the list of tools from utils.py

In [1]:
from utils import *

iminuit version: 2.31.3


The code snippet below gives an example of how to open the data file (in the root format) with `uproot` and convert the content to a pandas dataframe. Each entry in the data file contains the total energy $E$, the carthesian momentum components ($p_x, p_y, p_z$) as well as the charge $Q$ of the two detected muons. 

(You might find the following [uproot tutorial](https://hsf-training.github.io/hsf-training-uproot-webpage/01-introduction/index.html) useful) 

In [2]:
# Open rootfile containing CMS muons and put into pandas dataframe
path = "cms_muons.root"
df = None
with uproot.open(path) as f:
    tree = f["Events"]         # replace with your TTree name

    # mu1 is a muon, mu2 is an anti-muon (Q means electric charge)
    cols = ["mu1_px","mu1_py","mu1_pz","mu1_E","mu1_Q",
            "mu2_px","mu2_py","mu2_pz","mu2_E","mu2_Q"]

    df = tree.arrays(cols, library="pd")   # → pandas DataFrame for later use


## Question 3 (10 marks)

The code example below shows how to construct a Lorentz vector from the energy and momentum components of a particle and how to add the transverse momentum, $p_T$, to the dataframe.

- Calculate the invariant mass of the muon pair, $M_{\mu^{+}\mu^{-}}$ as well as the angle $\cos\theta^*$ and add them to the dataframe.
- Plot both variables for all events in the dataframe.

**Here and in the following: make sure you calculate and display statistical uncertainties for all the histogram bins in your plots.**


In [3]:

'''
Example using the vector object to extract arrays of Lorentz vectors for each muon, 
but you don't have to use this. More information here: 
https://vector.readthedocs.io/en/latest/src/object.html
in particular https://vector.readthedocs.io/en/latest/src/momentum4d.html
'''
mu1_lv = vector.array({"px":df["mu1_px"].to_numpy(), "py":df["mu1_py"].to_numpy(), "pz":df["mu1_pz"].to_numpy(),"E":df["mu1_E"].to_numpy()})
mu2_lv = vector.array({"px":df["mu2_px"].to_numpy(), "py":df["mu2_py"].to_numpy(), "pz":df["mu2_pz"].to_numpy(),"E":df["mu2_E"].to_numpy()})

df["pt1"] = mu1_lv.pt
df["pt2"] = mu2_lv.pt

## Question 4 (10 marks)

Now that we have calculated the angle $\cos\theta^*$, 

- plot the forward-backward asymmetry $A_{FB} = \frac{N_F - N_B}{N_F + N_B}$, where $N_F$ ($N_B$) are the number of events with $\cos\theta^* >0$ ($\cos\theta^* <0$), as a function of the di-muon invariant mass. 

Use the following range and bin boundaries in your histogram for the di-muon invariant mass (same as in CMS paper): 60,70,78,84,87,89,91,93,95,98,104,112,120.

## Question 5 (5 marks)

The forward-backward asymmetry $A_{FB}$ is sensitive to the rapidity $Y$ of the di-muon system.

- Calculate the rapidity ${Y}_{\mu^{+}\mu^{-}}$ of the **di-muon system** for each event, add it to the dataframe and make a plot of this distribution.

(The rapidity is defined as $Y = \frac{1}{2} \ln \frac{E + p_z}{E - p_z}$ but should be directly available as a property of a four-vector.)


## Question 6 (20 marks)

Now split your dataset into different regions of absolute value of rapidity $|Y_{\mu^+\mu^{-}}|$ in bins of width 0.4, ranging from 0.0 to 2.4.

- Plot $\cos\theta^*$ for each of the six rapidity bins. 

- Also plot $A_{FB}$ as a function of the di-muon invariant mass for each of these six $|Y_{\mu^+\mu^{-}}|$ ranges. Use the mass binning given in Question 4.

- Briefly comment on what you observe.

(Make sure you consider and add statistical uncertainties in each histogram bin and propagate uncertainties into derived quantities where necessary.)



## Question 7 (30 marks)

We now have the data manipulated such that we can extract the parameter of interest, $\sin^2\theta_W$.
Before we can fit the theoretical expression for $A_{FB} (M)$ from Question 2 to the data to extract $\sin^2\theta_W$ we need to consider that our definition of the di-lepton rapidity "relies on the fact that on average the dileptons are boosted in the quark direction" (see CMS paper). In addition, imperfect acceptance and detection efficiency for the muons mean that the measured $A_{FB} (M)$ has a different shape compared to the true $A_{FB} (M)$.

We therefore need to correct the true $A_{FB}$ from Question 6 for these effects. This is done for you by the function `compute_afb_correction` in `utils.py` and provided to you in the code snippet below. It also shows you how to apply this correction to the 
$A_{FB} (M)$ in the function `afb`.

- The aim of this question is to write some code to simultaneously fit the corrected expression for $A_{FB}(M)$ to the six histograms of $A_{FB}(M)$ in the different $|Y_{\mu^+\mu^{-}}|$ ranges and use $\sin^2\theta_W$ as a free parameter whose value is determined by the fit.
  - To start with, we need to convert the continuous expression for $A_{FB}(M)$ to a binned version to match the data histograms.
  - Comment on why in the example below we rate average the $A_{FB}(M)$.
  - With the rate averaged binned and corrected expression for $A_{FB}(M)$ now write some code to simultaneously fit the corrected expression for $A_{FB}(M)$ to the six histograms of $A_{FB}(M)$ in the different $|Y_{\mu^+\mu^{-}}|$ ranges. As mentioned in the example below, use `LeastSquares` as cost function for Minuit.


In [10]:
'''
The snippet below defines the expected AFB value as a function
of sinThetaW**2 and the invariant mass of the dimuon system.
of the invariant mass of the dimuon system. It applies a correction
that accounts for the sculpting of the predicted AFB due to detection
efficiency effects. 

It then passes the product of the predicted AFB and Z-boson lineshape
to a wrapper function that bins it in dilepton invariant mass.

The Minuit cost function you should minimise is the so-called LeastSquares

cost = LeastSquares(x, data["afb_vals"], data["afb_errs"], afb_rate_avg_binned,name=names)

where: 
    x is the function variable ie the dimuon invariant mass 
    data["afb_vals"] is an array containing the central values of the measured AFB in bins of dimuon invariant mass
    data["afb_errs"] is an array containing the error values of the measured AFB in bins of dimuon invariant mass
    afb_rate_avg_binned is the binned prediction of afb that depends on sinThetaW**2 
    names = ["stwsq"]

You can then follow the steps in the Example or Coursework-1 to run the minimisation, fit for sinThetaW**2 
and make a plot of the result

'''

# define various constants for the afb computation
mZ = 91.1876 # GeV
gZ = 2.4955  # GeV
GF = 1.1663787e-5  # GeV^-2
K  = 8./127*np.sqrt(2)*np.pi/(GF*mZ**2)
af = {"mu": -0.5, "u": 0.5, "d":-0.5}
Qf = {"mu": -1, "u": 2/3, "d":-1/3}

# compute the afb correction
afb_corr = compute_afb_correction() 

# define the afb function, continuous and binned
def afb(x,stwsq):
       
    Dm = 1.0-mZ**2/x**2
    def vf(t):
        return af[t]-2*Qf[t]*stwsq
    def afb_q(t):
        num = 6*af["mu"]*af[t]*(8.*vf("mu")*vf(t)-Qf[t]*K*Dm)
        den = 16.*(vf("mu")**2+af["mu"]**2)*(vf(t)**2+af[t]**2)-8.*vf("mu")*vf(t)*Qf[t]*K*Dm+\
              Qf[t]**2*K**2*(Dm**2+gZ**2/mZ**2)
        return num/den
  
    return (0.5*afb_q("u")+ 0.5*afb_q("d"))*afb_corr(x)

# define the afb function multiplied by the Z lineshape 
def afb_rate_avg(x, stwsq):    
    return afb(x, stwsq)*z_lineshape(x)
# wrapper function that computes binned  Z lineshape-average to pass to minimiser
def afb_rate_avg_binned(x, *params, bin_edges=bins):
    return func_rate_avg_binned(afb_rate_avg, x, *params, bin_edges=bins)


## Question 8 (5 marks)
If we look carefully at the CMS paper we realize that the quantity we have actually measured is $\sin^2\theta_{\rm eff}^{\rm lept}$, which is called the effective leptonic weak mixing angle!

- Briefly discuss the difference between $\sin^2\theta_{\rm eff}^{\rm lept}$ and $\sin^2\theta_W$.
- Compare the value and uncertainties you obtained with the literature value for $\sin^2\theta_{\rm eff}^{\rm lept}$.

## Question 9 (10 marks)

- Based on information in the CMS paper, briefly list the dominant systematic uncertainties and describe how they affect the measurement of $\sin^2\theta_{\rm eff}^{\rm lept}$.

## End of Coursework
